In [1]:
%matplotlib inline

from glob import glob
from itertools import combinations
import json
import os
import sys

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from scipy.stats import spearmanr

from tqdm import tqdm

pd.set_option("display.max_columns", 101)
pd.set_option("display.float_format", lambda x: "%.2f" % x )

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src', 'data')
sys.path.append(src_dir)

# import my method from the source code
%aimport utils
%aimport download_planet_lib

# load environment varibles
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

pass

/Users/bull/anaconda/envs/farmdrive/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Geographic Features

For our first pass at the data, our area of interest is just **Nakuru** county.

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('postgresql://localhost/farmdrive')
session = sessionmaker(bind=engine)()

In [3]:
county_name = 'Nakuru'
crop_table = 'maiz_p--ssa'
crop_name = 'maize'

In [4]:
# individual geojson polygons for each raster pixel
query = """
SELECT
json_build_object(
    'type', 'Feature',
    'id', (poly_pixels.x || '_' || poly_pixels.y),
    'geometry', ST_AsGeoJSON(1, poly_pixels.geom, 15, 2) :: JSON,
    'properties', json_build_object('{crop_name}_yield', poly_pixels.val)
)
FROM
  (SELECT (ST_PixelAsPolygons(ST_Union(ST_Clip("{crop_table}".rast, clipped_geom.geom)))).*
    FROM
      "{crop_table}",
      (SELECT county.geom FROM county WHERE county.county = '{county_name}') AS clipped_geom
    WHERE ST_Intersects("{crop_table}".rast, clipped_geom.geom)
  ) AS poly_pixels;
"""

query = query.format(crop_name=crop_name,
           crop_table=crop_table,
           county_name=county_name)

# Execute the query in the session
result = session.execute(query)

# for now, just get the first result. ultimately, we'll need them all.
aoi_raster = result.fetchall()
aoi_raster = aoi_raster[2]

geojson_from_postgis = aoi_raster['json_build_object']
geojson_from_postgis

{'geometry': {'coordinates': [[[35.9166783844273, 0.083326218967615],
    [36.0000117291493, 0.083326218967615],
    [36.0000117291493, -7.12575437e-06],
    [35.9166783844273, -7.12575437e-06],
    [35.9166783844273, 0.083326218967615]]],
  'crs': {'properties': {'name': 'EPSG:4326'}, 'type': 'name'},
  'type': 'Polygon'},
 'id': '8_3',
 'properties': {'maize_yield': 1},
 'type': 'Feature'}

In [5]:
geojson_from_postgis['geometry']['coordinates'][0] = geojson_from_postgis['geometry']['coordinates'][0][::-1]

In [6]:
from subprocess import check_output, CalledProcessError, STDOUT

planet_data = os.path.join(os.pardir,
                             'data',
                             'raw',
                             'planet')

county_data = os.path.join(planet_data, county_name)

county_pixel_dir = os.path.join(county_data,
                                geojson_from_postgis['id'])

asset_dir = os.path.join(county_data,
                         'assets')

os.makedirs(county_pixel_dir, exist_ok=True)

geojson_input = os.path.join(county_pixel_dir, 'geojson_epsg4326.geojson')
geojson_output = os.path.join(county_pixel_dir, 'geojson_epsg32637.geojson')

with open(geojson_input, 'w') as gj_file:
    json.dump(geojson_from_postgis, gj_file)

try:
    check_output(['ogr2ogr',
                  '-f',
                  'GeoJSON',
                  geojson_output,
                  '-t_srs',
                  'EPSG:32637',
                  geojson_input], stderr=STDOUT)

except CalledProcessError as e:
    print(e.output)

# Download Data

In [7]:
geo_json_geometry = geojson_from_postgis['geometry']

# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

# MAIZE harvest season in Kenya is Aug - Oct
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-07-31T00:00:00.000Z",
    "lte": "2016-10-31T00:00:00.000Z"
  }
}

# filter any images which are more than 10% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.05
  }
}

# create a filter that combines our geo and date filters
maize_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [8]:
def has_local_scene(scene_id, asset_type):
    glob_path = os.path.join(asset_dir, '{}_{}.tif'.format(sid, asset_type))
    scene_paths = glob(glob_path)    
    return len(scene_paths) > 0

In [9]:
import time

# Product type, PSOrthoTile or REOrthoTile
search_type = 'PSOrthoTile'

# visual or analytic - which asset type to download
asset_type = 'analytic'


# get the planet scenes for our query
scene_ids = download_planet_lib.run_search({'item_types': [search_type],
                                        'filter': maize_filter})

# check for scenes that we _don't_ already have
not_local_scene_ids = []
for sid in scene_ids:
    if not has_local_scene(sid, asset_type):
        not_local_scene_ids.append(sid)
    
# activate all of the planet scenes that we may want
res2 = download_planet_lib.process_activation(download_planet_lib.activate,
                                          not_local_scene_ids,
                                          search_type,
                                          asset_type)

for i in tqdm(range(100)):
    resie = download_planet_lib.process_activation(download_planet_lib.check_activation,
                                               not_local_scene_ids,
                                               search_type,
                                               asset_type)
    
    if all(resie):
        print('Activated!')
        break
    else:
        time.sleep(15)

results = download_planet_lib.process_download(asset_dir,
                                           not_local_scene_ids,
                                           search_type,
                                           asset_type,
                                           True)

results
assert all(results)

Running query


  0%|          | 0/100 [00:00<?, ?it/s]

Activated!


# Merge tiles for each pixel and crop to json

In [10]:
import math
import warnings

import numpy as np

from rasterio import windows
from rasterio.transform import Affine


def my_merge(sources, bounds=None, res=None, nodata=None, precision=7):
    first = sources[0]
    first_res = first.res
    nodataval = first.nodatavals[0]
    dtype = first.dtypes[0]

    # Extent from option or extent of all inputs.
    if bounds:
        dst_w, dst_s, dst_e, dst_n = bounds
    else:
        # scan input files.
        xs = []
        ys = []
        for src in sources:
            left, bottom, right, top = src.bounds
            xs.extend([left, right])
            ys.extend([bottom, top])
        dst_w, dst_s, dst_e, dst_n = min(xs), min(ys), max(xs), max(ys)

    output_transform = Affine.translation(dst_w, dst_n)

    # Resolution/pixel size.
    if not res:
        res = first_res
    elif not np.iterable(res):
        res = (res, res)
    elif len(res) == 1:
        res = (res[0], res[0])
    output_transform *= Affine.scale(res[0], -res[1])

    # Compute output array shape. We guarantee it will cover the output
    # bounds completely.
    output_width = int(math.ceil((dst_e - dst_w) / res[0]))
    output_height = int(math.ceil((dst_n - dst_s) / res[1]))

    # Adjust bounds to fit.
    dst_e, dst_s = output_transform * (output_width, output_height)

    # create destination array
    dest = np.zeros((first.count, output_height, output_width), dtype=dtype)

    if nodata is not None:
        nodataval = nodata

    if nodataval is not None:
        # Only fill if the nodataval is within dtype's range.
        inrange = False
        if np.dtype(dtype).kind in ('i', 'u'):
            info = np.iinfo(dtype)
            inrange = (info.min <= nodataval <= info.max)
        elif np.dtype(dtype).kind == 'f':
            info = np.finfo(dtype)
            inrange = (info.min <= nodataval <= info.max)
        if inrange:
            dest.fill(nodataval)
        else:
            warnings.warn(
                "Input file's nodata value, %s, is beyond the valid "
                "range of its data type, %s. Consider overriding it "
                "using the --nodata option for better results." % (
                    nodataval, dtype))
    else:
        nodataval = 0

    for src in tqdm(sources):
        # Real World (tm) use of boundless reads.
        # This approach uses the maximum amount of memory to solve the problem.
        # Making it more efficient is a TODO.

        # 1. Compute spatial intersection of destination and source.
        src_w, src_s, src_e, src_n = src.bounds

        int_w = src_w if src_w > dst_w else dst_w
        int_s = src_s if src_s > dst_s else dst_s
        int_e = src_e if src_e < dst_e else dst_e
        int_n = src_n if src_n < dst_n else dst_n

        # 2. Compute the source window.
        src_window = windows.from_bounds(
            int_w, int_s, int_e, int_n, src.transform,
            boundless=True, precision=precision)

        # 3. Compute the destination window.
        dst_window = windows.from_bounds(
            int_w, int_s, int_e, int_n, output_transform,
            boundless=True, precision=precision)


        # 4. Initialize temp array.
        tcount = first.count
        trows, tcols = tuple(b - a for a, b in dst_window)

        temp_shape = (tcount, trows, tcols)

        temp = np.zeros(temp_shape, dtype=dtype)
        temp = src.read(out=temp, window=src_window, boundless=False,
                        masked=True)

        # 5. Copy elements of temp into dest.
        roff, coff = dst_window[0][0], dst_window[1][0]

        region = dest[:, roff:roff + trows, coff:coff + tcols]
        np.copyto(
            region, temp,
            where=np.logical_and(region == nodataval, temp.mask == False))

    return dest, output_transform

In [27]:
import logging
logger = logging.getLogger("rasterio.merge")
logger.setLevel(logging.DEBUG)

# logging.Logger.manager.loggerDict

In [10]:
from glob import glob

import rasterio
from rasterio import features
from rasterio.merge import merge

paths = [glob(county_data + os.sep + 'assets' + os.sep + '{}_{}.tif'.format(sid, asset_type))[0] \
    for sid in scene_ids]

planet_crs = {'proj': 'utm', 'zone': 37, 'ellps': 'WGS84', 'datum': 'WGS84', 'units':'m', 'no_defs': True}
srcs = [rasterio.open(p, crs=planet_crs) for p in paths]

with open(geojson_output) as gj_file:
    reprojgeoj = json.load(gj_file)

out_image, out_transform = merge(srcs, bounds=features.bounds(reprojgeoj))

# save the resulting raster
out_meta = srcs[0].meta.copy()
out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform})

raster_merged_path = os.path.join(county_pixel_dir, geojson_from_postgis['id'] + '.tif')

with rasterio.open(raster_merged_path, "w", **out_meta) as dest:
    dest.write(out_image)

ValueError: negative dimensions are not allowed

In [16]:
a = srcs[0]
a.affine
a#.bounds

/Users/bull/anaconda/envs/farmdrive/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: DeprecationWarning: 'src.affine' is deprecated.  Please switch to 'src.transform'. See https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


<open DatasetReader name='../data/raw/planet/Nakuru/assets/273250_3739001_2016-10-21_0c76_analytic.tif' mode='r'>

# Crop to pixels

Now done using bounds above.

In [ ]:
# import gdal

# # Crop back to AOI
# tif_loc = '../data/raw/planet/nakuru/11_1/271486_3739102_2016-10-19_0c65_analytic.tif'
# output_file = os.path.splitext(tif_loc)[0] + '_clipped.tif'

# import json
# with open('../test.geojson', 'w') as tg:
#     json.dump(aoi_raster['json_build_object'], tg)

# # GDAL Warp crops the image by our AOI, and saves it
# gdal.Warp(output_file,
#           tif_loc,
#           srcSRS = 'EPSG:32637',
# #           dstSRS = 'WGS64',
#           format='GTiff',
#           dstAlpha=True,
#           cutlineDSName='../test2.geojson',
#           cropToCutline=True)

In [ ]:
# import rasterio
# from rasterio.mask import mask

# # Crop back to AOI
# tif_loc = '../data/raw/planet/nakuru/11_1/11_rio.tif'
# output_file = os.path.splitext(tif_loc)[0] + '_clipped.tif'

# with open('../test2.geojson') as tg:
#     reprojgeoj = json.load(tg)

# planet_crs = {'proj': 'utm', 'zone': 37, 'ellps': 'WGS84', 'datum': 'WGS84', 'units':'m', 'no_defs': True}
# # load the raster, mask it by the polygon and crop it
# with rasterio.open(tif_loc, crs=planet_crs) as src:
#     out_image, out_transform = mask(src,
#                                     [reprojgeoj],
#                                     crop=True)
    
#     #plt.imshow(out_image)
#     out_meta = src.meta.copy()

# # save the resulting raster  
# out_meta.update({"driver": "GTiff",
#     "height": out_image.shape[1],
#     "width": out_image.shape[2],
#     "transform": out_transform})

# with rasterio.open(output_file, "w", **out_meta) as dest:
#     dest.write(out_image)

In [ ]:
from itertools import compress

test = ['a', 'b', 'c']
mask = [True, False, True]

list(compress(test, mask))